# Read healthcare.gov CSVs

This notebook reads the four CSV files located in `data/healthcare.gov` and displays the first five rows of each file.

In [2]:
# Read and display first five rows of the four healthcare.gov CSVs
from pathlib import Path
import pandas as pd
from IPython.display import display

base = Path("data/healthcare.gov")
files = {
    "benefits_and_cost_sharing": "benefits-and-cost-sharing-puf.csv",
    "plan_attributes": "Plan_Attributes_PUF.csv",
    "rate": "Rate_PUF.csv",
    "service_area": "service-area-puf.csv",
}

for key, fname in files.items():
    path = base / fname
    print(f"=== {fname} ===")
    if not path.exists():
        print(f"File not found: {path}\n")
        continue
    try:
        df = pd.read_csv(path, low_memory=False)
        display(df.head(5))
    except Exception as e:
        print(f"Error reading {path}: {e}\n")

# -------------------------------------------------
# Continued analysis specifically for Rate_PUF.csv
# -------------------------------------------------
rate_path = base / files['rate']
if rate_path.exists():
    try:
        print("\n=== Analysis: Rate_PUF.csv ===")
        df_rate = pd.read_csv(rate_path, low_memory=False)

        # Basic overview
        print("Shape:", df_rate.shape)
        print("\nFirst 5 rows:")
        display(df_rate.head(5))

        # Info (prints to stdout in notebooks)
        print("\nInfo:")
        df_rate.info()

        # Missing values (percent)
        print("\nMissing values (percent):")
        missing = (df_rate.isna().mean() * 100).sort_values(ascending=False)
        display(missing[missing > 0].head(20))

        # Numeric summary
        numcols = df_rate.select_dtypes(include='number').columns.tolist()
        if numcols:
            print("\nNumeric summary (describe):")
            display(df_rate[numcols].describe().T)
        else:
            print("\nNo numeric columns found.")

        # Categorical top values
        objcols = df_rate.select_dtypes(include='object').columns.tolist()
        if objcols:
            print("\nTop value counts for object columns (showing up to 10):")
            for col in objcols:
                vc = df_rate[col].value_counts(dropna=False)
                print(f"\nColumn: {col} — unique={vc.size}")
                display(vc.head(10))
        else:
            print("\nNo object (categorical) columns found.")

        # Try to find any 'rate'-like numeric columns and show top rows
        rate_like = [c for c in df_rate.columns if 'rate' in c.lower()]
        if rate_like:
            print(f"\nFound rate-like columns: {rate_like}")
            for c in rate_like:
                if c in numcols:
                    print(f"\nTop 5 rows by {c}:")
                    display(df_rate.nlargest(5, columns=c)[[c] + df_rate.columns[:4].tolist()])
                else:
                    print(f"\nColumn {c} found but it's not numeric; showing value counts:")
                    display(df_rate[c].value_counts(dropna=False).head(10))
        else:
            print("\nNo column name containing 'rate' found; showing top 3 numeric columns by max value instead:")
            if numcols:
                maxvals = {c: df_rate[c].max() for c in numcols}
                sorted_cols = sorted(maxvals, key=maxvals.get, reverse=True)[:3]
                for c in sorted_cols:
                    print(f"\nTop 5 by {c} (max={maxvals[c]}):")
                    display(df_rate.nlargest(5, columns=c)[[c]])

        # Sample rows with most numeric data present
        if numcols:
            numeric_nonnull = df_rate[numcols].notnull().sum(axis=1)
            print('\nSample 5 rows with highest count of non-null numeric values:')
            display(df_rate.loc[numeric_nonnull.nlargest(5).index].head())

    except Exception as e:
        print(f"Error analyzing {rate_path}: {e}")
else:
    print(f"Rate file not found at {rate_path}")


=== benefits-and-cost-sharing-puf.csv ===


,BusinessYear,StateCode,IssuerId,SourceName,ImportDate,StandardComponentId,PlanId,BenefitName,CopayInnTier1,CopayInnTier2,...,IsEHB,IsCovered,QuantLimitOnSvc,LimitQty,LimitUnit,Exclusions,Explanation,EHBVarReason,IsExclFromInnMOOP,IsExclFromOonMOOP
0,2025,AK,21989,HIOS,2024-08-29 01:02:15,21989AK0030001,21989AK0030001-00,Accidental Dental,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025,AK,21989,HIOS,2024-08-29 01:02:15,21989AK0030001,21989AK0030001-00,Basic Dental Care - Adult,Not Applicable,NaN,...,NaN,Covered,Yes,1100.0,Dollars per Year,NaN,See policy for other limits,Not EHB,Yes,Yes
2,2025,AK,21989,HIOS,2024-08-29 01:02:15,21989AK0030001,21989AK0030001-00,Basic Dental Care - Child,Not Applicable,NaN,...,Yes,Covered,NaN,NaN,NaN,NaN,See policy for limits,Additional EHB Benefit,No,No
3,2025,AK,21989,HIOS,2024-08-29 01:02:15,21989AK0030001,21989AK0030001-00,Dental Check-Up for Children,Not Applicable,NaN,...,Yes,Covered,NaN,NaN,NaN,NaN,See policy for limits,Additional EHB Benefit,No,No
4,2025,AK,21989,HIOS,2024-08-29 01:02:15,21989AK0030001,21989AK0030001-00,Major Dental Care - Adult,Not Applicable,NaN,...,NaN,Covered,Yes,1100.0,Dollars per Year,NaN,See policy for other limits,Not EHB,Yes,Yes


=== Plan_Attributes_PUF.csv ===


,BusinessYear,StateCode,IssuerId,IssuerMarketPlaceMarketingName,SourceName,ImportDate,MarketCoverage,DentalOnlyPlan,StandardComponentId,PlanMarketingName,...,TEHBDedOutOfNetFamilyPerPerson,TEHBDedOutOfNetFamilyPerGroup,TEHBDedCombInnOonIndividual,TEHBDedCombInnOonFamilyPerPerson,TEHBDedCombInnOonFamilyPerGroup,IsHSAEligible,HSAOrHRAEmployerContribution,HSAOrHRAEmployerContributionAmount,URLForSummaryofBenefitsCoverage,PlanBrochure
0,2025,AK,21989,Delta Dental of Alaska,HIOS,2024-08-29 01:02:15,Individual,Yes,21989AK0030001,Delta Dental Premier Plan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.deltadentalak.com/-/media/deltaden...
1,2025,AK,21989,Delta Dental of Alaska,HIOS,2024-08-29 01:02:15,Individual,Yes,21989AK0030001,Delta Dental Premier Plan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.deltadentalak.com/-/media/deltaden...
2,2025,AK,21989,Delta Dental of Alaska,HIOS,2024-08-29 01:02:15,Individual,Yes,21989AK0050001,Delta Dental PPO 1000 Plan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.deltadentalak.com/-/media/deltaden...
3,2025,AK,21989,Delta Dental of Alaska,HIOS,2024-08-29 01:02:15,Individual,Yes,21989AK0050001,Delta Dental PPO 1000 Plan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.deltadentalak.com/-/media/deltaden...
4,2025,AK,21989,Delta Dental of Alaska,HIOS,2024-08-29 01:02:15,Individual,Yes,21989AK0050002,Delta Dental PPO 1500 Plan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.deltadentalak.com/-/media/deltaden...


=== Rate_PUF.csv ===


...

,IndividualRate,BusinessYear,StateCode,IssuerId,SourceName
124226,9999.0,2025,AZ,86830,HIOS
124227,9999.0,2025,AZ,86830,HIOS
124228,9999.0,2025,AZ,86830,HIOS
124229,9999.0,2025,AZ,86830,HIOS
124230,9999.0,2025,AZ,86830,HIOS



Top 5 rows by IndividualTobaccoRate:


,IndividualTobaccoRate,BusinessYear,StateCode,IssuerId,SourceName
2404619,5877.03,2025,WV,50328,SERFF
2399009,5863.64,2025,WV,50328,SERFF
2404874,5817.39,2025,WV,50328,SERFF
2399264,5804.14,2025,WV,50328,SERFF
2404618,5783.00,2025,WV,50328,SERFF



Sample 5 rows with highest count of non-null numeric values:


,BusinessYear,StateCode,IssuerId,SourceName,ImportDate,RateEffectiveDate,RateExpirationDate,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,IndividualTobaccoRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents
510,2025,AK,21989,HIOS,2024-08-29 01:02:15,2025-07-01,2025-09-30,21989AK0080001,Rating Area 1,NaN,Family Option,54.45,NaN,113.26,130.69,130.69,130.69,194.95,194.95,194.95
511,2025,AK,21989,HIOS,2024-08-29 01:02:15,2025-04-01,2025-06-30,21989AK0080001,Rating Area 1,NaN,Family Option,53.79,NaN,111.88,129.10,129.10,129.10,192.57,192.57,192.57
512,2025,AK,21989,HIOS,2024-08-29 01:02:15,2025-01-01,2025-03-31,21989AK0080001,Rating Area 1,NaN,Family Option,53.13,NaN,110.50,127.50,127.50,127.50,190.20,190.20,190.20
513,2025,AK,21989,HIOS,2024-08-29 01:02:15,2025-10-01,2025-12-31,21989AK0080001,Rating Area 1,NaN,Family Option,55.12,NaN,114.65,132.28,132.28,132.28,197.32,197.32,197.32
514,2025,AK,21989,HIOS,2024-08-29 01:02:15,2025-10-01,2025-12-31,21989AK0080001,Rating Area 2,NaN,Family Option,52.36,NaN,108.92,125.67,125.67,125.67,187.46,187.46,187.46


推荐只使用：

IndividualRate

IndividualTobaccoRate

Age

RatingArea

Plan attributes（来自 Plan PUF）

家庭字段可以在报告中注明 “not widely used in ACA pricing”